In [42]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.models import model_from_json


In [43]:
data = []
labels = []

bears = os.listdir('data/bear')
for bear in bears:
    if bear.endswith("jpg") or bear.endswith("png"):
        img = cv2.imread("data/bear/" + bear)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(0)
        
turkeys = os.listdir('data/turkey')
for turkey in turkeys:
    if turkey.endswith("jpg") or turkey.endswith("png"):
        img = cv2.imread("data/turkey/" + turkey)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(1)

    

In [44]:
animals = np.array(data)
labels = np.array(labels)
np.save('animals',animals)
np.save('labels',labels)
s = np.arange(animals.shape[0])
np.random.shuffle(s)
animals = animals[s]
labels = labels[s]
nclasses = len(np.unique(labels))
dlen = len(animals)



In [45]:
(x_train,x_test)=animals[(int)(0.1*dlen):],animals[:(int)(0.1*dlen)]
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

train_length=len(x_train)
test_length=len(x_test)

(y_train,y_test)=labels[(int)(0.1*dlen):],labels[:(int)(0.1*dlen)]
y_train=keras.utils.to_categorical(y_train,nclasses)
y_test=keras.utils.to_categorical(y_test,nclasses)


model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(600,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 6, 6, 64)          0         
__________

In [48]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [47]:
model.fit(x_train,y_train,batch_size=50,epochs=100,verbose=1)

Epoch 1/100
900/900 [==============================] - 3s 3ms/step - loss: 0.7091 - acc: 0.5589
Epoch 2/100
900/900 [==============================] - 1s 1ms/step - loss: 0.5704 - acc: 0.6533
Epoch 3/100
900/900 [==============================] - 1s 1ms/step - loss: 0.4618 - acc: 0.7778
Epoch 4/100
900/900 [==============================] - 1s 1ms/step - loss: 0.3347 - acc: 0.8722
Epoch 5/100
900/900 [==============================] - 1s 1ms/step - loss: 0.2348 - acc: 0.9111
Epoch 6/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1868 - acc: 0.9378
Epoch 7/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1656 - acc: 0.9333
Epoch 8/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1269 - acc: 0.9556
Epoch 9/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1042 - acc: 0.9656
Epoch 10/100
900/900 [==============================] - 1s 1ms/step - loss: 0.0849 - acc: 0.9733
Epoch 11/100
900/900 [=================

In [49]:
score = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test accuracy:', score[1])


99/99 [==============================] - 0s 3ms/step

 Test accuracy: 0.8787878787878788


In [50]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((50, 50))
    return np.array(image)

def get_animal_name(label):
    if label==0:
        return "bear"
    if label==1:
        return "turkey"
    return "Can't Detect"

def predict_animal(file):
    print("Predicting .................................")
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    animal=get_animal_name(label_index)
    print(animal)
    print("The predicted Animal is a "+animal+" with accuracy =    "+str(acc))    



In [52]:
predict_animal('data/test/01.jpg')


Predicting .................................
1/1 [==============================] - 0s 2ms/step
[[1.000000e+00 3.970076e-13]]
0
bear
The predicted Animal is a bear with accuracy =    1.0
